# Create Training Pickles

In [1]:
import pandas as pd
import imageio.v2 as imageio
import numpy as np
import pickle

In [3]:
classes = pd.read_csv("amanda_labeled_081023.csv")
classes

,filename,OBJECTID,id,areaverify,carea,eventid,expire,fcster,hailtag,issue,...,verify,wfo,windtag,year,ORIG_FID,radar_time,CWA,rid,top_class,sub_class
0,false_alarm_2007JAN1TOW1.png,7,2007JAN1TOW1,0,2420.790780,1,2007-01-05T05:38:00,CME,0.00,2007-01-05 05:11:00,...,0,JAN,0,2007,6,2007-01-05 05:00:00,JAN,2007JAN1TOW1,Disorganized and other,None
1,false_alarm_2007JAN4TOW1.png,11,2007JAN4TOW1,0,1813.991592,4,2007-01-05T08:45:00,CME,0.00,2007-01-05 08:06:00,...,0,JAN,0,2007,10,2007-01-05 08:00:00,JAN,2007JAN4TOW1,Linear,"QLCS, MCS"
2,false_alarm_2007FFC2TOW1.png,26,2007FFC2TOW1,0,1385.209944,2,2007-01-05T16:00:00,,0.00,2007-01-05 15:13:00,...,0,FFC,0,2007,25,2007-01-05 15:00:00,FFC,2007FFC2TOW1,Linear,"QLCS, MCS"
3,false_alarm_2007FFC3TOW1.png,27,2007FFC3TOW1,0,1309.517400,3,2007-01-05T16:00:00,,0.00,2007-01-05 15:27:00,...,0,FFC,0,2007,26,2007-01-05 15:00:00,FFC,2007FFC3TOW1,Linear,"QLCS, MCS"
4,false_alarm_2007FFC4TOW1.png,28,2007FFC4TOW1,0,518.929777,4,2007-01-05T16:00:00,,0.00,2007-01-05 15:36:00,...,0,FFC,0,2007,27,2007-01-05 16:00:00,FFC,2007FFC4TOW1,Linear,"QLCS, MCS"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,false_alarm_2017TBW13TOW1.png,31832,2017TBW13TOW1,0,5213.737125,13,2017-09-10T12:10:00,TBW,0.75,2017-09-10 11:57:00,...,0,TBW,0,2017,31831,2017-09-10 12:00:00,TBW,2017TBW13TOW1,Tropical,None
4120,false_alarm_2017TBW15TOW1.png,31837,2017TBW15TOW1,0,5213.737125,15,2017-09-10T15:15:00,TBW,0.75,2017-09-10 14:45:00,...,0,TBW,0,2017,31836,2017-09-10 15:00:00,TBW,2017TBW15TOW1,Tropical,None
4121,false_alarm_2017TBW16TOW1.png,31839,2017TBW16TOW1,0,11472.104470,16,2017-09-10T15:30:00,TBW,0.75,2017-09-10 15:08:00,...,0,TBW,0,2017,31838,2017-09-10 15:00:00,TBW,2017TBW16TOW1,Tropical,None
4122,false_alarm_2017MPX42TOW1.png,31929,2017MPX42TOW1,0,5444.839357,42,2017-09-24T21:45:00,TDK,0.75,2017-09-24 21:18:00,...,0,MPX,0,2017,31928,2017-09-24 21:00:00,MPX,2017MPX42TOW1,Multicell,multicell clusters


In [4]:
classes = classes[(classes['top_class'] != 'BLANK')]
classes['top_class'].unique()

array(['Disorganized and other', 'Linear', 'Multicell', 'Discrete',
       'Tropical'], dtype=object)

In [5]:
class_lookup = ['Disorganized and other','Linear','Multicell', 'Discrete', 'Tropical']

In [6]:
import numpy as np
import pickle

prefix = "/home/scratch/gridrad_cref/villa/patches/false_alarms"

# Create the various subsets of data for each of the training datasets
for subset, name in zip([(2007, 2014), (2015, 2016), (2017, 2018)], ("train", "validation", "test")):
    sub_ = classes[(classes.year >= subset[0]) & (classes.year <= subset[1])].copy()
    image_data = []

    # Iterate through each row of the dataframe 
    for rid, row in sub_.iterrows():

        # Open and append each image
        fname = f"{prefix}/{row.radar_time[5:7]}/{row.wfo}/{row.filename}"   
        img = imageio.imread(fname, pilmode='P')
        image_data.append(img)

        # Expand the dimensions and save the labels
        imgs = np.expand_dims(np.array(image_data), axis=3)
        classes_ = sub_['top_class'].values
        class_codes = [np.where(np.array(class_lookup) == classes_[x])[0][0] for x in range(len(classes_))]

        # Dump to a pickle file for later use
        pickle.dump([imgs, class_codes], open("/home/scratch/jcorner1/Thesis/data/nexrad/{}_{}_{}.pkl".format(subset[0], subset[1], name), "wb"))